In [4]:
import pandas as pd
import numpy as np
import glob; import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import datetime
import seaborn as sns
import copy
import requests

from sklearn.cluster import KMeans
# from sklearn.metrics import accuracy_score

from matplotlib import pyplot as plt
import random as rand

In [5]:
df = pd.read_csv("data_preprocessing_v1.csv",index_col = 0)
df_info = pd.read_csv("data_info.csv",index_col=0)

df_new_info = pd.read_csv("./new_complete_data/EPSIS_발전설비_발전기별.csv",index_col=0)


pv_plant_data = pd.read_csv("data_site_info.csv",index_col=0)
pv_plant_data.head()

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,사업명,주 소,위경도,설비용량(kW),거래형태
번호,,,,,
1,수도공고(본관),서울시 강남구 개포로 410,"37.4850307, 127.0639073",99.00,PPA
2,대전제일고,대전시 서구 배재로 46길 77,"36.3133199,127.3670431",99.00,PPA
3,세광고(본관),충북 청주시 서원구 2순환로 1572,"36.6031204,127.4507651",190.40,PPA
4,세광고(수학관),충북 청주시 서원구 2순환로 1572,"36.6031204,127.4507651",30.60,PPA
5,세광중,충북 청주시 서원구 2순환로 1574,"36.6029202,127.4527515",99.96,PPA


In [6]:
df_new_info['발전기명']

회사명
(주)명보태양광발전소    (주)명보태양광발전소
GS POWER          월미태양광발전소
나노켐태양광발전소        나노켐태양광발전소
동원에너지                동원에너지
선영 태양광발전소        선영 태양광발전소
                  ...     
힘찬태양광발전소 6호    힘찬태양광발전소 6호
힘찬태양광발전소 7호    힘찬태양광발전소 7호
힘찬태양광발전소 8호    힘찬태양광발전소 8호
힘찬태양광발전소 9호    힘찬태양광발전소 9호
NaN                     통합
Name: 발전기명, Length: 73024, dtype: object

# 기상데이터 저장 및 결측구간 존재여부 확인

In [17]:
address = ""
address_data = "./new_complete_data/complete_data_20192101_20201124/"
site="new_ALL"

data_Weather = pd.read_csv(address+'OBS_ASOS_TIM_'+site+'.csv',encoding='CP949')

place_names = data_Weather["place"].values
info_place_names = np.unique(place_names)
print(info_place_names)


for i,name in enumerate(info_place_names):
    
    data_each = data_Weather[data_Weather["place"]==name]
    data_each.to_csv(address+'OBS_ASOS_TIM_new_'+name+'.csv',encoding='utf-8-sig')
    print("Weahter data of "+name+" Saved.")

incomplete_data_name = []
incomplete_data_len = []
for i,name in enumerate(info_place_names):
    
    data_each = data_Weather[data_Weather["place"]==name]
    if len(data_each)!=8640:
        print("Weahter data of "+name+" is not 멀쩡.")
        print("Length is "+str(len(data_each))+".")
        incomplete_data_name.append(name)
        incomplete_data_len.append(len(data_each))
    
print(incomplete_data_name)
print(incomplete_data_len)

['강릉' '강진군' '강화' '거제' '거창' '경주시' '고산' '고창' '고창군' '고흥' '광양시' '광주' '구미' '군산'
 '금산' '김해시' '남원' '남해' '대관령' '대구' '대전' '동두천' '동해' '목포' '문경' '밀양' '백령도' '보령'
 '보성군' '보은' '봉화' '부산' '부안' '부여' '북강릉' '북창원' '북춘천' '산청' '상주' '서귀포' '서산'
 '서울' '성산' '세종' '속초' '수원' '순창군' '순천' '안동' '양산시' '양평' '여수' '영광군' '영덕' '영월'
 '영주' '영천' '완도' '울릉도' '울산' '울진' '원주' '의령군' '의성' '이천' '인제' '인천' '임실' '장수'
 '장흥' '전주' '정선군' '정읍' '제주' '제천' '진도군' '진주' '창원' '천안' '철원' '청송군' '청주' '추풍령'
 '춘천' '충주' '태백' '통영' '파주' '포항' '함양군' '합천' '해남' '홍성' '홍천' '흑산도']
Weahter data of 강릉 Saved.
Weahter data of 강진군 Saved.
Weahter data of 강화 Saved.
Weahter data of 거제 Saved.
Weahter data of 거창 Saved.
Weahter data of 경주시 Saved.
Weahter data of 고산 Saved.
Weahter data of 고창 Saved.
Weahter data of 고창군 Saved.
Weahter data of 고흥 Saved.
Weahter data of 광양시 Saved.
Weahter data of 광주 Saved.
Weahter data of 구미 Saved.
Weahter data of 군산 Saved.
Weahter data of 금산 Saved.
Weahter data of 김해시 Saved.
Weahter data of 남원 Saved.
Weahter data of 남해 Saved.
Weahter data of 대관령 

In [19]:
main_site_name =info_place_names

# 기상데이터 결측 여부 확인 함수

In [20]:
main_site_name =info_place_names
def check_miss_weather():
    
    data_com = data_Weather[data_Weather["place"]=="흑산도"]
    complete_time_array = data_com["Date"].values
    time_len = len(complete_time_array)
    complete_date_array = np.zeros(time_len,dtype='object')
    complete_hour_array = np.zeros(24,dtype='object')

    for i in range(time_len):
        complete_date_array[i] = complete_time_array[i][:10]

    for i in range(24):
        complete_hour_array[i] = complete_time_array[i][10:]

    complete_date_array=np.unique(complete_date_array)
    complete_hour_array=np.unique(complete_hour_array)
#     print(complete_date_array)
#     print(complete_hour_array)
    
    for k in range(len(incomplete_data_name)):
        data_incomplete = data_Weather[data_Weather["place"]==incomplete_data_name[k]]
#         print(len(data_incomplete))

        incomplete_time_array = data_incomplete["Date"].values
        time_len = len(incomplete_time_array)
#         print(time_len)
        incomplete_date_array = np.zeros(time_len,dtype='object')
        
        for i in range(time_len):
            incomplete_date_array[i] = incomplete_time_array[i][:10]

        incomplete_date_array=np.unique(incomplete_date_array)

        # print(incomplete_date_array)
        print("-"*40)
        print("-"*40)
        print("지역 : "+incomplete_data_name[k])
        for i in range(len(complete_time_array)):
            if complete_time_array[i] not in incomplete_time_array:

                print(complete_time_array[i]+"없음.")
#             if np.isin(incomplete_time_array,complete_time_array[i],assume_unique=True):
#                 print(complete_time_array[i]+"없음.")

# 기상데이터 결측 처리하는 함수(Linear Interpolation)

In [21]:
def impute_miss_weather():

    imputed_data = []
    
    
    data_com = data_Weather[data_Weather["place"]=="흑산도"]
    complete_time_array = data_com["Date"].values
    time_len = len(complete_time_array)
    complete_date_array = np.zeros(time_len,dtype='object')
    complete_hour_array = np.zeros(24,dtype='object')

    for i in range(time_len):
        complete_date_array[i] = complete_time_array[i][:10]

    for i in range(24):
        complete_hour_array[i] = complete_time_array[i][10:]

    complete_date_array=np.unique(complete_date_array)
    complete_hour_array=np.unique(complete_hour_array)
    #     print(complete_date_array)
    #     print(complete_hour_array)
    
    
    
    for k in range(len(incomplete_data_name)):
    
        data_incomplete = data_Weather[data_Weather["place"]==incomplete_data_name[k]]
        incomplete_time_array = data_incomplete["Date"].values
        time_len = len(incomplete_time_array)
        incomplete_date_array = np.zeros(time_len,dtype='object')

        for i in range(time_len):
            incomplete_date_array[i] = incomplete_time_array[i][:10]

        incomplete_date_array=np.unique(incomplete_date_array)

        print("-"*40)
        print("-"*40)
        print("지역 : "+incomplete_data_name[k])
        for i in range(len(complete_time_array)):
            if complete_time_array[i] not in incomplete_time_array:

                print(complete_time_array[i]+"없음."+"\t해당 index : "+str(data_incomplete.index[i]),i)

                info_row = data_incomplete.values[0]
                info_row[2] = complete_time_array[i]
                info_row[3] = np.nan
                info_row[4] = np.nan
                info_row[5] = np.nan
                info_row[6] = np.nan
                empty_row_values = [info_row]
                print(empty_row_values)
                data_incomplete = pd.DataFrame(np.insert(data_incomplete.values, i, values=empty_row_values, axis=0),columns=["Location","place","Date","Temp","Wind","Humid","Cloud"])
        imputed_data.append(data_incomplete)
    return imputed_data

In [22]:
check_miss_weather()

----------------------------------------
----------------------------------------
지역 : 강릉
----------------------------------------
----------------------------------------
지역 : 강진군
----------------------------------------
----------------------------------------
지역 : 강화
----------------------------------------
----------------------------------------
지역 : 거제
----------------------------------------
----------------------------------------
지역 : 거창
----------------------------------------
----------------------------------------
지역 : 경주시
----------------------------------------
----------------------------------------
지역 : 고산
----------------------------------------
----------------------------------------
지역 : 고창
----------------------------------------
----------------------------------------
지역 : 고창군
----------------------------------------
----------------------------------------
지역 : 고흥
----------------------------------------
----------------------------------------
지역 : 광양시
------

----------------------------------------
----------------------------------------
지역 : 해남
----------------------------------------
----------------------------------------
지역 : 홍성
----------------------------------------
----------------------------------------
지역 : 홍천
----------------------------------------
----------------------------------------
지역 : 흑산도


In [23]:
imputed_weather_data=impute_miss_weather()

----------------------------------------
----------------------------------------
지역 : 강릉
----------------------------------------
----------------------------------------
지역 : 강진군
----------------------------------------
----------------------------------------
지역 : 강화
----------------------------------------
----------------------------------------
지역 : 거제
----------------------------------------
----------------------------------------
지역 : 거창
----------------------------------------
----------------------------------------
지역 : 경주시
----------------------------------------
----------------------------------------
지역 : 고산
----------------------------------------
----------------------------------------
지역 : 고창
----------------------------------------
----------------------------------------
지역 : 고창군
----------------------------------------
----------------------------------------
지역 : 고흥
----------------------------------------
----------------------------------------
지역 : 광양시
------

----------------------------------------
----------------------------------------
지역 : 파주
----------------------------------------
----------------------------------------
지역 : 포항
----------------------------------------
----------------------------------------
지역 : 함양군
----------------------------------------
----------------------------------------
지역 : 합천
----------------------------------------
----------------------------------------
지역 : 해남
----------------------------------------
----------------------------------------
지역 : 홍성
----------------------------------------
----------------------------------------
지역 : 홍천
----------------------------------------
----------------------------------------
지역 : 흑산도


In [24]:
def save_missed_weather_dataset():
    for i in range(len(imputed_weather_data)):
        name = incomplete_data_name[i]
        data_each = imputed_weather_data[i]
        data_each.to_csv(address+'OBS_ASOS_TIM_new_'+name+'.csv',encoding='utf-8-sig')
        print("Weahter data of "+name+" Saved.")
        
save_missed_weather_dataset()

Weahter data of 강릉 Saved.
Weahter data of 강진군 Saved.
Weahter data of 강화 Saved.
Weahter data of 거제 Saved.
Weahter data of 거창 Saved.
Weahter data of 경주시 Saved.
Weahter data of 고산 Saved.
Weahter data of 고창 Saved.
Weahter data of 고창군 Saved.
Weahter data of 고흥 Saved.
Weahter data of 광양시 Saved.
Weahter data of 광주 Saved.
Weahter data of 구미 Saved.
Weahter data of 군산 Saved.
Weahter data of 금산 Saved.
Weahter data of 김해시 Saved.
Weahter data of 남원 Saved.
Weahter data of 남해 Saved.
Weahter data of 대관령 Saved.
Weahter data of 대구 Saved.
Weahter data of 대전 Saved.
Weahter data of 동두천 Saved.
Weahter data of 동해 Saved.
Weahter data of 목포 Saved.
Weahter data of 문경 Saved.
Weahter data of 밀양 Saved.
Weahter data of 백령도 Saved.
Weahter data of 보령 Saved.
Weahter data of 보성군 Saved.
Weahter data of 보은 Saved.
Weahter data of 봉화 Saved.
Weahter data of 부산 Saved.
Weahter data of 부안 Saved.
Weahter data of 부여 Saved.
Weahter data of 북강릉 Saved.
Weahter data of 북창원 Saved.
Weahter data of 북춘천 Saved.
Weahter data of 산청 Saved.


In [25]:
def save_nan_imputed_weather_dataset():
    for i,name in enumerate(info_place_names):
        
        loaded_imputed_weather_data = pd.read_csv(address+'OBS_ASOS_TIM_new_'+name+'.csv',index_col=0)
        
        loaded_imputed_weather_data["Temp"]=loaded_imputed_weather_data["Temp"].astype("float64").interpolate()
        loaded_imputed_weather_data["Wind"]=loaded_imputed_weather_data["Wind"].astype("float64").interpolate()
        loaded_imputed_weather_data["Humid"]=loaded_imputed_weather_data["Humid"].astype("float64").interpolate()
        loaded_imputed_weather_data["Cloud"]=loaded_imputed_weather_data["Cloud"].astype("float64").interpolate()
        data_each = loaded_imputed_weather_data.reset_index(drop=True)
        data_each.to_csv(address+'OBS_ASOS_TIM_new_'+name+'_imputed.csv',encoding='utf-8-sig')

        print("Weahter data of "+name+" Saved.")
        

In [26]:
save_nan_imputed_weather_dataset()

Weahter data of 강릉 Saved.
Weahter data of 강진군 Saved.
Weahter data of 강화 Saved.
Weahter data of 거제 Saved.
Weahter data of 거창 Saved.
Weahter data of 경주시 Saved.
Weahter data of 고산 Saved.
Weahter data of 고창 Saved.
Weahter data of 고창군 Saved.
Weahter data of 고흥 Saved.
Weahter data of 광양시 Saved.
Weahter data of 광주 Saved.
Weahter data of 구미 Saved.
Weahter data of 군산 Saved.
Weahter data of 금산 Saved.
Weahter data of 김해시 Saved.
Weahter data of 남원 Saved.
Weahter data of 남해 Saved.
Weahter data of 대관령 Saved.
Weahter data of 대구 Saved.
Weahter data of 대전 Saved.
Weahter data of 동두천 Saved.
Weahter data of 동해 Saved.
Weahter data of 목포 Saved.
Weahter data of 문경 Saved.
Weahter data of 밀양 Saved.
Weahter data of 백령도 Saved.
Weahter data of 보령 Saved.
Weahter data of 보성군 Saved.
Weahter data of 보은 Saved.
Weahter data of 봉화 Saved.
Weahter data of 부산 Saved.
Weahter data of 부안 Saved.
Weahter data of 부여 Saved.
Weahter data of 북강릉 Saved.
Weahter data of 북창원 Saved.
Weahter data of 북춘천 Saved.
Weahter data of 산청 Saved.


In [34]:
def visualize_seq(time_seq,show_len):
  time = [i for i in range(show_len*time_seq.shape[1])]
  print(time_seq.shape)
  rand_idx = rand.randrange(len(time_seq)-show_len)
  rand_idx = 0
  print("Random Index :",rand_idx)
  plt.plot(time,np.reshape(time_seq[rand_idx:rand_idx+show_len],-1))
  plt.title("Visualization")
  plt.xlabel('Time')
  plt.ylabel('Load')
  plt.show()

In [36]:
def count_missing(input_seq):
    num_missing = 0
    series= input_seq
    zero_series=np.zeros(24,dtype="float32")
    missing_mask=np.zeros(num_days,dtype="float32")
    for i,one_day in enumerate(input_seq):
        if (one_day==zero_series).all() :
            num_missing += 1
            missing_mask[i]=0
        else:
            missing_mask[i]=1
    return num_missing, missing_mask

In [37]:
def count_consecutive_missing(input_seq):
    num_missing = 0
    series= input_seq
    zero_series=np.zeros(24,dtype="float32")
    for i,one_day in enumerate(input_seq):
        if (one_day==zero_series).all() :
            num_missing += 1
    return num_missing

In [38]:
def missing_check(data_mat):
    num_check_users = data_mat.shape[0]
    num_check_days = data_mat.shape[1]
    missing_mat = np.zeros((num_check_users,num_check_days),dtype="float32")
    missing_rate=np.zeros(data_mat.shape[0])
    
    for i,user in enumerate(data_mat):
        if i==5:
            print(i,"번째 user")
        missing_rate[i],missing_mat[i]=count_missing(data_mat[i])
    #     print(i,missing_rate[i])
    missing_rate = missing_rate *100/ num_check_days
    return missing_mat,missing_rate